In [ ]:
# Importing modules
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from pyproj import CRS
import geojson
import folium


In [ ]:
# Reading the data into a dataframe
data = pd.read_json('https://iot.fvh.fi/opendata/uiras/uiras-meta.json', 
                    orient="index") 

# Creating shapely Point objects from the coordinates
data['geometry'] = data.apply(lambda row: 
                              Point(row['lon'], row['lat']), axis=1)

# Creating a geodataframe
geo = gpd.GeoDataFrame(data, 
                       geometry='geometry')

# Setting the CRS to EPSG:3857
geo = geo.set_crs(epsg=4326)
geo = geo.to_crs(epsg=3857)


In [ ]:
geo

In [ ]:
# Dropping the Uunisaari data since there is no data
geo = geo.drop(index="003C62A8")


In [ ]:
geo

In [ ]:
# Reading the temperature data
temps = pd.read_csv(
    "https://iot.fvh.fi/opendata/uiras/uiras-all-data.csv",
    parse_dates=['readable_time'],
    usecols=["readable_time", "temp_out1", "dev-id"]
)


In [ ]:
temps

In [ ]:
# Reversing the dataframe
temps = temps[::-1]


In [ ]:
temps.head(20)

In [ ]:
# Creating an empty list and dataframe
latest_temps = pd.DataFrame()
devices = []

# For-looping the temps dataframe
for index, row in temps.iterrows():
    
    # Checking if device-id has been already stored in the list
    if row["dev-id"] not in devices:

        # Appending rows to the new dataframe
        latest_temps = latest_temps.append(row)
        
        # Adding the device-id to the list
        devices.append(row["dev-id"])
    
    # Since I know there are only 14 devices,
    # no reason to iterate over the whole dataset
    if len(devices) == 14:
        break


In [ ]:
devices

In [ ]:
# Creating a string from the timestamp object
latest_temps["time"] = latest_temps.readable_time.dt.strftime('%d-%m-%Y %H:%M')


In [ ]:
latest_temps

In [ ]:
# Removing the unnecessary columns
latest_temps = latest_temps[["dev-id", "temp_out1", "time"]]

geo = geo[["name", "geometry"]]

In [ ]:
geo

In [ ]:
# Merging the datasets
join = geo.merge(latest_temps, left_index=True, right_on="dev-id")


In [ ]:
join

In [ ]:
join.crs

In [ ]:
import folium

  
# Defining tooltip
my_tooltip = folium.features.GeoJsonTooltip(
    fields=["name", "time", "temp_out1"],
    aliases=["Place:", "Time of measurement", "Temperature (°C)"],
    labels=True,
    sticky=False
)

# This popup doesnt work, maybe due to a bug
my_popup = folium.features.GeoJsonPopup(
    fields=["name", "time", "temp_out1"],
    aliases=["Place:", "Measured at:", "Temperature (°C):"],
    localize=True,
    parse_html=True
)

# Defining the point data
point_gjson = folium.features.GeoJson(
    join,
    name="Beaches",
    # There are some issues with changing the icon when dealing with GeoJson
   # icon=folium.Icon(color="green", icon="umbrella-beach", prefix="fa"),
    popup=my_popup,
    tooltip=my_tooltip
)

# Defining map instance
m = folium.Map(
    location=[60.25, 24.95],
    zoom_start=10,
    control_scale=True,
    tiles="OpenStreetMap"
)

# Adding the point data to the map
point_gjson.add_to(m)

loc = 'Water Temperatures in real-time by HRI, plotted by Justus Poutanen'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc) 

m.get_root().html.add_child(folium.Element(title_html))

m

In [ ]:
# Saving the map to html
m.save("Water_temps.html")
